# BTC Feature Engineering

## Overview
This notebook creates technical indicators and features for the BTC prediction model:
- Technical indicators (RSI, MACD, Moving Averages, Ichimoku)
- Price-based features (OHLCV derivatives)
- Target variable creation (Sell signal detection)
- Feature selection and preprocessing

## Technical Indicators to Implement
1. **RSI (Relative Strength Index)**: Momentum oscillator
2. **MACD (Moving Average Convergence Divergence)**: Trend following
3. **Moving Averages**: SMA, EMA for different periods
4. **Ichimoku Cloud**: Comprehensive trend analysis
5. **Bollinger Bands**: Volatility indicators
6. **Volume indicators**: Volume-based features

## Target Variable
- **Sell Signal**: -15% price drop within 30-day window
- **Price Calculation**: Current close → lowest LOW price
- **Priority**: SELL > BUY > WAIT (first seen rule)


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import talib
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
